In [ ]:
# Based on code from the following link:
# https://towardsdatascience.com/multi-label-classification-using-bert-roberta-xlnet-xlm-and-distilbert-with-simple-transformers-b3e0cda12ce5

In [ ]:
#!pip install simpletransformers==0.34.4

In [ ]:
# import keras
# keras.__version__

In [ ]:
# import tensorflow
# tensorflow.__version__

In [ ]:
# import torch
# torch.__version__

In [ ]:
# !jupyter --version

In [ ]:
# !pip install scikit-learn==0.23.1

In [ ]:
# !pip install pandas==1.0.5

In [ ]:
# !pip show simpletransformers 

In [ ]:
# import sklearn
# sklearn.__version__

In [1]:
import pandas as pd
#from simpletransformers.classification import MultiLabelClassificationModel
import ast

In [2]:
from simpletransformers.experimental.classification import MultiLabelClassificationModel

# Model Training

In [3]:
df = pd.read_csv('./train.tsv', sep='\t')

In [ ]:
df.labels = df.labels.apply(eval)

In [ ]:
model = MultiLabelClassificationModel('bert', "emilyalsentzer/Bio_ClinicalBERT", num_labels=124, 
                                      args={'train_batch_size':4,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True})

In [ ]:
model.train_model(df)

# Sliding Window Experimental Model Training

In [1]:
import pandas as pd
#from simpletransformers.classification import MultiLabelClassificationModel
import ast
from simpletransformers.experimental.classification import MultiLabelClassificationModel

df = pd.read_csv('./train.tsv', sep='\t')
df.labels = df.labels.apply(eval)

In [2]:
model = MultiLabelClassificationModel('bert', "emilyalsentzer/Bio_ClinicalBERT", num_labels=124, 
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           'tensorboard_dir':'./outputs/',
                                           'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

In [3]:
model.train_model(df)

Running loss: 0.724157

C:\Users\brent\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: -0.383459

C:\Users\brent\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: -48.270176


Running loss: -68.9806068


Running loss: -57.6811602



# Model Evaluation

In [6]:
import pandas as pd
#from simpletransformers.classification import MultiLabelClassificationModel
import ast
from simpletransformers.experimental.classification import MultiLabelClassificationModel
import sklearn
import numpy as np

In [2]:
model = MultiLabelClassificationModel('bert', "./outputs/checkpoint-95931-epoch-3", num_labels=124,
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           'tensorboard_dir':'./outputs/',
                                           'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

In [3]:
eval_df = pd.read_csv('./test.tsv', sep='\t')
eval_df.labels = eval_df.labels.apply(eval)

In [4]:
predictions, raw_outputs = model.predict(eval_df['text'])

In [9]:
y_true = np.array(eval_df['labels'].tolist())
y_true[y_true > 1] = 1

for cut_off in np.linspace(0.1,1,9,endpoint=False):  

    y_pred = np.empty((0,124))

    for i in range(len(raw_outputs)):
        temp = np.max(raw_outputs[i], axis=0)
        temp[temp>cut_off] = 1
        temp[temp<=cut_off] = 0
        y_pred = np.vstack([y_pred,temp])
    
    print("\n\nCalculated F-scores for cutoff:", cut_off)
    print("\nMacro:", sklearn.metrics.f1_score(y_true, y_pred, average='macro'))
    print("Micro:", sklearn.metrics.f1_score(y_true, y_pred, average='micro'))
    print("Weighted:", sklearn.metrics.f1_score(y_true, y_pred, average='weighted'))
    print("Samples:",sklearn.metrics.f1_score(y_true, y_pred, average='samples', zero_division=1))
    print("None:\n",sklearn.metrics.f1_score(y_true, y_pred, average=None))



Calculated F-scores for cutoff: 0.1

Macro: 0.1742618199926649
Micro: 0.44548401401545196
Weighted: 0.5129617311212284
Samples: 0.4404989012223472
None:
 [0.84465825 0.83954097 0.81695389 0.65557316 0.60609388 0.42431762
 0.28475034 0.75722294 0.54137931 0.48145933 0.44047619 0.84465825
 0.60435835 0.52771392 0.59107291 0.         0.20014144 0.31687106
 0.24687933 0.6593674  0.5382193  0.60609388 0.83954097 0.
 0.13107511 0.         0.17410072 0.31193881 0.         0.
 0.29090909 0.         0.08308157 0.         0.1641953  0.
 0.         0.         0.         0.         0.         0.14812112
 0.         0.18129703 0.         0.         0.         0.
 0.         0.         0.         0.57175014 0.         0.
 0.         0.         0.         0.         0.36732068 0.
 0.         0.         0.         0.         0.38446849 0.
 0.         0.         0.41176471 0.         0.         0.
 0.         0.         0.         0.16389892 0.         0.
 0.13102687 0.09741476 0.         0.         



Calculated F-scores for cutoff: 0.7000000000000001

Macro: 0.07095989742381462
Micro: 0.5157977219936754
Weighted: 0.34690605339460784
Samples: 0.5123730101719025
None:
 [0.90419806 0.83954097 0.81695389 0.         0.         0.
 0.         0.75722294 0.         0.         0.         0.84465825
 0.60435835 0.         0.59107291 0.         0.         0.
 0.         0.6593674  0.         0.60609388 0.83954097 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.      

# Compare to 2 Epoch Model

In [10]:
model = MultiLabelClassificationModel('bert', "./outputs/checkpoint-63954-epoch-2", num_labels=124,
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           'tensorboard_dir':'./outputs/',
                                           'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

In [11]:
predictions, raw_outputs = model.predict(eval_df['text'])

In [12]:
y_true = np.array(eval_df['labels'].tolist())
y_true[y_true > 1] = 1

for cut_off in np.linspace(0.1,1,9,endpoint=False):  

    y_pred = np.empty((0,124))

    for i in range(len(raw_outputs)):
        temp = np.max(raw_outputs[i], axis=0)
        temp[temp>cut_off] = 1
        temp[temp<=cut_off] = 0
        y_pred = np.vstack([y_pred,temp])
    
    print("\n\nCalculated F-scores for cutoff:", cut_off)
    print("\nMacro:", sklearn.metrics.f1_score(y_true, y_pred, average='macro'))
    print("Micro:", sklearn.metrics.f1_score(y_true, y_pred, average='micro'))
    print("Weighted:", sklearn.metrics.f1_score(y_true, y_pred, average='weighted'))
    print("Samples:",sklearn.metrics.f1_score(y_true, y_pred, average='samples', zero_division=1))
    print("None:\n",sklearn.metrics.f1_score(y_true, y_pred, average=None))



Calculated F-scores for cutoff: 0.1

Macro: 0.1732386101703209
Micro: 0.45128354725787634
Weighted: 0.5125896828306503
Samples: 0.4456942534521216
None:
 [0.84465825 0.83954097 0.81695389 0.65557316 0.60609388 0.42431762
 0.28475034 0.75722294 0.54137931 0.48145933 0.44047619 0.84465825
 0.60435835 0.52771392 0.59107291 0.         0.20014144 0.31687106
 0.2479564  0.6491018  0.5382193  0.60609388 0.83954097 0.
 0.13107511 0.         0.17410072 0.31193881 0.         0.
 0.29090909 0.         0.         0.         0.1641953  0.
 0.         0.         0.         0.         0.         0.14812112
 0.         0.18129703 0.11581292 0.         0.         0.
 0.         0.         0.         0.57175014 0.         0.
 0.         0.         0.         0.         0.36732068 0.
 0.         0.         0.         0.         0.38446849 0.
 0.         0.         0.41176471 0.         0.         0.
 0.         0.         0.         0.16389892 0.         0.
 0.13102687 0.         0.         0.         



Calculated F-scores for cutoff: 0.7000000000000001

Macro: 0.07082170234603283
Micro: 0.5156101381972807
Weighted: 0.3468217060142614
Samples: 0.5122318229932645
None:
 [0.90419806 0.83954097 0.81695389 0.         0.         0.
 0.         0.75722294 0.         0.         0.         0.84465825
 0.60435835 0.         0.59107291 0.         0.         0.
 0.         0.6491018  0.         0.60609388 0.83954097 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.       

# Compare to 1 Epoch Model

In [13]:
model = MultiLabelClassificationModel('bert', "./outputs/checkpoint-31977-epoch-1", num_labels=124,
                                      args={'train_batch_size':4,
                                            'sliding_window': True,
                                            'stride': 0.8,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True,
                                            'no_cache': True,
                                            'multiprocessing_chunksize': 410,
                                           'tensorboard_dir':'./outputs/',
                                           'wandb_project': False,
                                           'save_model_every_epoch': True,
                                           'no_save': False,
                                           'save_optimizer_and_scheduler': True})

In [14]:
predictions, raw_outputs = model.predict(eval_df['text'])

In [15]:
y_true = np.array(eval_df['labels'].tolist())
y_true[y_true > 1] = 1

for cut_off in np.linspace(0.1,1,9,endpoint=False):  

    y_pred = np.empty((0,124))

    for i in range(len(raw_outputs)):
        temp = np.max(raw_outputs[i], axis=0)
        temp[temp>cut_off] = 1
        temp[temp<=cut_off] = 0
        y_pred = np.vstack([y_pred,temp])
    
    print("\n\nCalculated F-scores for cutoff:", cut_off)
    print("\nMacro:", sklearn.metrics.f1_score(y_true, y_pred, average='macro'))
    print("Micro:", sklearn.metrics.f1_score(y_true, y_pred, average='micro'))
    print("Weighted:", sklearn.metrics.f1_score(y_true, y_pred, average='weighted'))
    print("Samples:",sklearn.metrics.f1_score(y_true, y_pred, average='samples', zero_division=1))
    print("None:\n",sklearn.metrics.f1_score(y_true, y_pred, average=None))



Calculated F-scores for cutoff: 0.1

Macro: 0.1757971450486654
Micro: 0.446947332794314
Weighted: 0.5164590768777718
Samples: 0.44343468159506666
None:
 [0.90419806 0.83954097 0.81695389 0.6550449  0.60609388 0.42431762
 0.28475034 0.75722294 0.54137931 0.48145933 0.44047619 0.84465825
 0.60435835 0.52771392 0.59107291 0.         0.20014144 0.31687106
 0.2553802  0.67919799 0.5382193  0.60609388 0.83954097 0.
 0.13107511 0.         0.17410072 0.31193881 0.         0.
 0.29090909 0.         0.08308157 0.         0.1641953  0.
 0.         0.         0.         0.         0.         0.14812112
 0.         0.18129703 0.         0.         0.         0.
 0.         0.         0.         0.57175014 0.         0.
 0.         0.         0.         0.         0.36732068 0.
 0.         0.         0.         0.         0.38446849 0.
 0.         0.         0.41176471 0.         0.         0.
 0.         0.         0.         0.16389892 0.         0.
 0.13102687 0.09741476 0.         0.         0



Calculated F-scores for cutoff: 0.7000000000000001

Macro: 0.07651056516609162
Micro: 0.5319649521337011
Weighted: 0.3661163474468814
Samples: 0.5248919290443675
None:
 [0.90419806 0.83954097 0.81695389 0.6550449  0.         0.
 0.         0.75722294 0.         0.         0.         0.84465825
 0.60435835 0.         0.59107291 0.         0.         0.
 0.         0.67919799 0.         0.60609388 0.83954097 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.       